<a href="https://colab.research.google.com/github/Sobiii/Search-Engine-Design-G4/blob/Sobii/Sobs_Attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extracting Documents

## CISI ALL extraction

In [72]:
import os as os

# get the current working directory
notebook_dir = os.getcwd()

# define paths to CISI dataset files
cisi_all_path = os.path.join(notebook_dir, 'CISI.ALL')
cisi_qry_path = os.path.join(notebook_dir, 'CISI.QRY')
cisi_rel_path = os.path.join(notebook_dir, 'CISI.REL')

## Debugging round 1
changes:<br>
1) Alternate methods for extracting document ID[mistakenly done- can keep sabrinas version]<br>
2) Code able to extract author info correctly regardless of title lenght/ spanning 2 lines. Previous iteration contained .A tag before author list whenever a title was more than a line. This has been solved. Author lists can now also be more than 1 person.<br>
3) Code can now extract abstract information a little better than the previous iteration. Having multiple authors no longer results in the .W tag being extracted as part of the abstract.

### Improvements to point 3 still needed: abstract is incorrectly extracted if there are other tags such as .B still left in the document. Look at file 26 as an example. Unwanted tags will be extracted as part of the abstract unless removed. This leads to the accumilation of unwanted tags, names and dates in the abstract. Write code to remove unwanted tags from the source data or change approach to extracting abstract 


In [73]:
import re

def extraction(file_name_path):
    with open(file_name_path) as f:
        file_lines = f.readlines()

    doc_ids = []
    documents = []
    i = 0
    while i < len(file_lines):
        if file_lines[i].startswith(".I"):
            doc_id = file_lines[i].split()[1]
            doc_ids.append(doc_id)
            if file_lines[i+1].startswith(".T") and file_lines[i+3].startswith(".A"):
                title = file_lines[i+2][:].strip()
            else:
                title = file_lines[i+2].strip() + " " + file_lines[i+3].strip()
            authors = ""
            if any([line.startswith(".A") for line in file_lines[i+1:]]):
                author_start = i+4
                author_end = author_start
                while author_end < len(file_lines) and not any([file_lines[author_end].startswith(tag) for tag in [".T", ".B", ".W"]]):
                    author_end += 1
                authors = " ".join([file_lines[j].strip() for j in range(author_start, author_end)]).strip()
                authors = re.sub(r"^\s*\.\s*A\s*", "", authors)
                abstract_start = author_end
            else:
                abstract_start = i+4

            for t in range(i+4, len(file_lines)):
                if file_lines[t].startswith(".X"):
                    abstract_end = t
                    break
            else:
                abstract_end = len(file_lines)
            abstract = " ".join(file_lines[abstract_start:abstract_end]).strip().replace('\n', ' ')
            abstract = re.sub(r"^\s*\.\s*W\s*", "", abstract)
            abstract = " ".join(abstract.split())
            documents.append({"document_id": doc_id, "title": title, "authors": authors, "abstract": abstract})
            i = abstract_end
        else:
            i += 1

    return documents



In [ ]:
documents = extraction(cisi_all_path)
documents

## CISI QRY extraction

The following code is correct and works for the first 57 queries (1-57) how wever the remaining queiries have a different format and thus different code to extract those. Work on that next.

1) Possible approach is to split the CISI.QRY file into 2 and work with two different qry files. I think this is the easiet option. If all else fails just stick to the first files of 57.

# Code for first 57 Queries

In [75]:
# query extraction for the CISI.QRY file
def extraction_query(file_name_path):
    with open(file_name_path) as f:
        file_lines = f.readlines()

    queries = []
    i = 0
    while i < len(file_lines):
        if file_lines[i].startswith(".I"):
            query_id = int(file_lines[i][3:])
            if file_lines[i+1].startswith(".W"):
                query_text = file_lines[i+2].strip()
                for t in range(i+3, len(file_lines)):
                    if file_lines[t].startswith(".I"):
                        break
                    query_text += " " + file_lines[t].strip()
            else:
                query_text = ""
            queries.append({"query_id": query_id, "query_text": query_text})
        i += 1

    return queries

In [ ]:
queries = extraction_query(cisi_qry_path)
queries

# Code for remaining queries

In [ ]:
# insert code